In [18]:
import transformers
from transformers import (
    AutoConfig, BertConfig, BertTokenizer, BertForRetrieval, BertForSequenceClassification)
import torch
import numpy as np

In [19]:
torch.cuda.is_available()
print(transformers.__version__)

2.11.0


In [13]:
config = AutoConfig.from_pretrained(
    "/home/transformers-public/output_dirs/classification_test/checkpoint-4500",
)
tokenizer = BertTokenizer.from_pretrained(
    "/home/transformers-public/output_dirs/classification_test/checkpoint-4500",
    do_lower_case=True,
)
model = BertForSequenceClassification.from_pretrained(
    "/home/transformers-public/output_dirs/classification_test/checkpoint-4500",
    from_tf=False,
    config=config,
)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [14]:
import pickle
pickle.dump(model, open("model.pkl", "wb"))

/home/sarahwooders_gmail_com/transformers/glisten/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [5]:
model = model.to(device)

In [6]:
max_seq_length=128
test_input = tokenizer.batch_encode_plus(
    [("test input", None)],
    max_length=max_seq_length,
    pad_to_max_length=True,
)

In [7]:
model_inp = dict(test_input)
model_inp['labels'] = [0]
# print(model_inp)
model_inp = {k: torch.tensor(v).to(device) for k, v in model_inp.items()}
model_out = model(**model_inp)

In [8]:
len(model_out)

3

In [9]:
print([x.shape for x in model_out])  # loss, logits, pooled_output (i.e. encoding)

[torch.Size([]), torch.Size([1, 30]), torch.Size([1, 768])]


In [10]:
predicted_logits = model_out[1].detach().cpu().numpy()
predicted_class = np.argmax(predicted_logits)

In [11]:
print(predicted_logits)

[[-1.3015641   2.0199366   2.3875945  -4.169917   -1.4164283  -4.2287807
  -1.8702354   0.35857558 -2.724265   -2.6567929  -1.8068223  -2.0170307
  -1.5032878  -2.942142    0.22073346 -0.8266337  -2.2023146   0.89124936
   2.5810018   0.01150025  0.69682723 -0.89142203  0.8963838  -0.4870002
   2.767781   -2.828165   -2.5763001  -0.83430356  0.36374697 -0.0487443 ]]


In [12]:
print(predicted_class)

24
